In [1]:
import sys
import os
import numpy as np
import multiprocessing
import dill
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


from util_functions import *
from uncertainty_analysis import *
from sim_helper_functions import *


module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path + "/src/simulations_v2")
from load_params import load_params

from multi_group_simulation import MultiGroupSimulation

In [2]:
NPOINTS = 13
MULT_UB = 3
MULT_LB = 1

In [3]:
# load Latin Hypercube Sampling results; change to the directory on your machine
lhs_output_sim_files = []
for i in range(2000):
    fname = '/home/yz685/group-testing/notebooks/apr_29_scenarios/point_{}.dill'.format(i)
    lhs_output_sim_files.append(fname)

scenario_data = load_sim_output(lhs_output_sim_files)
res_results = residential_regression(scenario_data)
res_pessimistic = calculate_pessimistic_scenario(res_results)
centre = get_centre_point()


The following function computes the new pessimistic configuration from the hyperplane calculation. It returns $\gamma$ and $x_{pess}$.

$$
\gamma = 1.645 \frac{\sqrt{c^T\Sigma c}\sqrt{c^T\Sigma^3c}}{c^T\Sigma^2c} 
= 1.645\frac{\sqrt{\sum_ic_i^2\sigma_i^2}{\sqrt{\sum_ic_i^2\sigma_i^6}}}{\sum_ic_i^2\sigma_i^4}.
$$

$$
x_{pess} = \mu +  1.645 \frac{\sqrt{c^T\Sigma c}\sqrt{c^T\Sigma^3c}}{c^T\Sigma^2c}\frac{\Sigma c}{\sqrt{c^T\Sigma c}} = \mu +  1.645 \frac{\sqrt{c^T\Sigma^3c}\Sigma c}{c^T\Sigma^2c}.
$$

One can directly compare $\gamma$ with 1.96 to see how differently the two pessimistic configurations proceed in the pessimistic direction.

The size of $\Sigma$ is proportional to the ranges $[LB, UB]$ we defined. Note that though the powers of $\Sigma$ cancel out in $\gamma$, i.e., we can use the range widths (or half-widths) as a proxy when calculating $\gamma$, the pessimistic direction vector $\frac{\Sigma c}{\sqrt{c^T\Sigma c}}$ does depend on the absolute size of $\Sigma$, and therefore depends on our assumption about the number of standard deviations in $[LB, UB]$.

In [4]:
def calculate_pessimistic_scenario_hyperplane(results, invquantile = 1.645, beta=1.96):
    # the keys in dict(results.params) specify whether this is for residential
    # or virtual vs. residential
    lr_results = dict(results.params)
    range_dict = dict()
    pess_direction = dict()
    params = set(lr_results.keys()) - set(['const'])
    for param in params:
        range_dict[param] = (PARAM_BOUNDS[param][1] - PARAM_BOUNDS[param][0])/2

    
    sum_squares_Sigma_1 = 0
    sum_squares_Sigma_2 = 0
    sum_squares_Sigma_3 = 0
    
    for param in params:
        sum_squares_Sigma_1 += (lr_results[param]*range_dict[param]) ** 2
        sum_squares_Sigma_2 += (lr_results[param]* range_dict[param]**2) ** 2
        sum_squares_Sigma_3 += (lr_results[param]* range_dict[param]**3) ** 2
    
    gamma = invquantile * np.sqrt(sum_squares_Sigma_1 * sum_squares_Sigma_3) / sum_squares_Sigma_2
    
    print('square root of c^T Sigma c = {}'.format(np.sqrt(sum_squares_Sigma_1)))
    
    for param in params:
        #pess_direction[param] = lr_results[param]*(range_dict[param])**2 / np.sqrt(sum_squares_Sigma_1)
        pess_direction[param] = lr_results[param]*(range_dict[param])**2 / np.sqrt(sum_squares_Sigma_1) / beta
    print('pessimistic direction {}'.format(pess_direction))
    
    
    # calculate pessimistic scenario based on available params
    pess_scenario = dict()
    old_pess_scenario = dict()
    for param in params:
        pess_scenario[param] = np.mean(PARAM_BOUNDS[param]) + gamma * pess_direction[param]
        old_pess_scenario[param] = np.mean(PARAM_BOUNDS[param]) + beta * pess_direction[param]
    
    # add default virtual params if not present
    default_virtual_param_vals = {param:(PARAM_BOUNDS[param][1] + PARAM_BOUNDS[param][0])/2 for param in ADDITIONAL_VIRTUAL_PARAMS}
    for virtual_param, val in default_virtual_param_vals.items():
        if virtual_param not in params:
            pess_scenario[virtual_param] = val

    return gamma, pess_direction, pess_scenario, old_pess_scenario

In [5]:
gamma, pess_direction, new_pess, old_pess = calculate_pessimistic_scenario_hyperplane(res_results)

square root of c^T Sigma c = 897.5026084324545
pessimistic direction {'ct_mult': -0.09375898267056981, 'inital_prev_mult': 0.026212744870253667, 'asymp_prob_mult': 0.0776580518826887, 'R0': 0.5232711930770082, 'test_noncompliance': 0.002637356803922309, 'outside_inf_mult': 0.012315513821857015, 'test_sensitivity': -0.0402855202074083, 'daily_self_report_prob': -0.006941788198258757, 'ID_time': 0.05110214294096922, 'E_time': -0.011170466880158828, 'Sy_time': -0.0012947192984487984, 'ct_testing_ratio': -0.0811794421475901}


In [6]:
# compare this to 1.96 (old)
gamma

2.207092840079398

Now we print the new_pess and old_pess values;
we can confirm the old_pess align with the ones computed using the old calculate_pessimistic_scenario(), which justifies including beta as a parameter

In [15]:
# print and compare current pessimistic value and new pessimistic value

print ("{:<30} {:<30} {:<30} {:<30}".format('PARAMETER', 'Old_Pess (old_gamma=1.96)', 'New_Pess (new_gamma=2.21)', 'Pessimistic_direction'))
  
for param, old_pess_value in old_pess.items():
    print ("{:<30} {:<30} {:<30} {:<30}".format(param, old_pess_value, new_pess[param], pess_direction[param]))

PARAMETER                      Old_Pess (old_gamma=1.96)      New_Pess (new_gamma=2.21)      Pessimistic_direction         
ID_time                        3.1001602001642996             3.1127871737977273             0.05110214294096922           
ct_mult                        1.3162323939656833             1.2930652206546571             -0.09375898267056981          
R0                             3.525611538430936              3.654908103660069              0.5232711930770082            
outside_inf_mult               1.0241384070908397             1.0271814823781196             0.012315513821857015          
daily_self_report_prob         0.34639409513141284            0.3446788289702754             -0.006941788198258757         
E_time                         1.9781058849148887             1.9753457425284573             -0.011170466880158828         
asymp_prob_mult                1.1522097816900698             1.1713985302847967             0.0776580518826887            
test_sen